<a href="https://colab.research.google.com/github/dariadementeva/NQ_Working/blob/main/mapillary_streetscapes_lux_city_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Installs the package.
!pip install mapillary

In [7]:
import mapillary.interface as mly
import requests
import json

In [19]:
from mapillary.utils.auth import Client

Client.set_token("MLY|25517726937837208|3f8625861dba933ed6b04631405be5bd")

In [20]:
mly.set_access_token(token)

{'token': 'SUCCESS'}

In [31]:
import math
import time
import random
import requests

ACCESS_TOKEN = "MLY|25517726937837208|3f8625861dba933ed6b04631405be5bd"

# Luxembourg City (rough) bbox
WEST, SOUTH, EAST, NORTH = 6.05, 49.55, 6.20, 49.67

FIELDS = "id,geometry,captured_at,thumb_1024_url"
LIMIT = 2000  # keep smaller than 2000 while stabilizing

def grid_bboxes(w, s, e, n, nx=4, ny=4):
    dx = (e - w) / nx
    dy = (n - s) / ny
    for ix in range(nx):
        for iy in range(ny):
            yield (w + ix*dx, s + iy*dy, w + (ix+1)*dx, s + (iy+1)*dy)

def get_images_bbox(bbox, retries=5):
    url = "https://graph.mapillary.com/images"
    params = {
        "access_token": ACCESS_TOKEN,
        "bbox": ",".join(map(str, bbox)),
        "fields": FIELDS,
        "limit": LIMIT,
    }

    for attempt in range(retries + 1):
        try:
            r = requests.get(url, params=params, timeout=60)
            # retry on server/rate-limit
            if r.status_code in (429, 500, 502, 503, 504):
                raise requests.HTTPError(f"{r.status_code} {r.text[:200]}", response=r)
            r.raise_for_status()
            return r.json()
        except Exception as ex:
            if attempt == retries:
                raise
            time.sleep((2 ** attempt) + random.random())

def paginate(first_json):
    out = []
    data = first_json
    while True:
        out.extend(data.get("data", []))
        nxt = data.get("paging", {}).get("next")
        if not nxt:
            break
        r = requests.get(nxt, timeout=60)
        if r.status_code in (429, 500, 502, 503, 504):
            # simple retry for paging, too
            time.sleep(2 + random.random())
            r = requests.get(nxt, timeout=60)
        r.raise_for_status()
        data = r.json()
    return out

all_images = []
for bbox in grid_bboxes(WEST, SOUTH, EAST, NORTH, nx=5, ny=5):
    js = get_images_bbox(bbox)
    imgs = paginate(js)
    all_images.extend(imgs)
    print("tile bbox", bbox, "->", len(imgs), "images")

# de-dup by id (grid overlaps can duplicate)
uniq = {img["id"]: img for img in all_images}
images = list(uniq.values())
print("TOTAL unique images:", len(images))

tile bbox (6.05, 49.55, 6.08, 49.574) -> 1892 images
tile bbox (6.05, 49.574, 6.08, 49.598) -> 1133 images
tile bbox (6.05, 49.598, 6.08, 49.622) -> 1536 images
tile bbox (6.05, 49.622, 6.08, 49.646) -> 1613 images
tile bbox (6.05, 49.646, 6.08, 49.67) -> 1874 images
tile bbox (6.08, 49.55, 6.11, 49.574) -> 1815 images
tile bbox (6.08, 49.574, 6.11, 49.598) -> 1944 images
tile bbox (6.08, 49.598, 6.11, 49.622) -> 1526 images
tile bbox (6.08, 49.622, 6.11, 49.646) -> 1604 images
tile bbox (6.08, 49.646, 6.11, 49.67) -> 1946 images
tile bbox (6.11, 49.55, 6.14, 49.574) -> 1898 images
tile bbox (6.11, 49.574, 6.14, 49.598) -> 1775 images
tile bbox (6.11, 49.598, 6.14, 49.622) -> 1865 images
tile bbox (6.11, 49.622, 6.14, 49.646) -> 1934 images
tile bbox (6.11, 49.646, 6.14, 49.67) -> 1938 images
tile bbox (6.14, 49.55, 6.17, 49.574) -> 1724 images
tile bbox (6.14, 49.574, 6.17, 49.598) -> 1795 images
tile bbox (6.14, 49.598, 6.17, 49.622) -> 1763 images
tile bbox (6.14, 49.622, 6.17, 49.6

In [32]:
images

[{'id': '135115332855948',
  'geometry': {'type': 'Point', 'coordinates': [6.0612534, 49.5592706]},
  'captured_at': 1680175272403,
  'thumb_1024_url': 'https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/An_aOcle52VSckWX-8Aj-80wq10VoFE98EFRqZVAWSs8ejWfz9v8dPCDXZ4yfwNHGXUN_Exh43v5Ol-c-DnM68MUduU4QwYhcEU-yKf4KgGeB0JX-fLzSj4B4o11vFk1KYT6B9H_nyT4qqhWfVVcpw?stp=s1024x576&edm=AOnQwmMEAAAA&_nc_gid=OCaOMKjbGr4CxLCNFvhJtA&_nc_oc=AdnUTMO1eWSaSSXKb1LOEQ7qsrmFpL5OUwDqf5L21v86b_qE9R7eILq7mxuM2egkXWE&ccb=10-5&oh=00_AfqsUtro9aKF3d2O38w8TS_7oo1FtvIhTqduyCwdHAffow&oe=69A2C415&_nc_sid=201bca'},
 {'id': '209139725534508',
  'geometry': {'type': 'Point',
   'coordinates': [6.061007700001, 49.559129100001]},
  'captured_at': 1698313506346,
  'thumb_1024_url': 'https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/An9jGl4-4YWV8dgGooydk8pocpLBSuiM-LnyPbKM0IizQxQGFv576RlY1qAI_7-1BefkhX_Prl1fbMkaYJQzMjFOCyde955uZ-4uPjK6bba5wEY46-QNAG4LxJZfV5UJc7DEtacYt-EUiVgM8JgsvYg?stp=s1024x768&edm=AOnQwmMEAAAA&_nc_gid=OCaOMKjbGr4CxLCNFvhJ

In [34]:
with open("mapillary_luxembourg_bbox.json", "w", encoding="utf-8") as f:
    json.dump(images, f, ensure_ascii=False, indent=2)

print("Saved", len(images), "records to mapillary_luxembourg_bbox.json")

Saved 43081 records to mapillary_luxembourg_bbox.json


In [38]:
import json
import os
import time
from urllib.parse import urlparse
import requests

ACCESS_TOKEN = "MLY|25517726937837208|3f8625861dba933ed6b04631405be5bd"
HEADERS = {"Authorization": f"OAuth {ACCESS_TOKEN}"}

INPUT_JSON = "mapillary_luxembourg_bbox.json"
OUT_DIR = "images"
SLEEP_SECONDS = 0.05

os.makedirs(OUT_DIR, exist_ok=True)

with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

if isinstance(data, dict):
    features = data.get("features", [])
elif isinstance(data, list):
    features = data
else:
    raise TypeError(f"Unexpected JSON root type: {type(data)}")

session = requests.Session()
session.headers.update(HEADERS)

def get_ext_from_url(url: str) -> str:
    path = urlparse(url).path
    _, ext = os.path.splitext(path)
    return ext if ext else ".jpg"

total = len(features)

for i, feat in enumerate(features, start=1):
    # support either {"properties": {...}} or direct dicts
    props = feat.get("properties", feat) if isinstance(feat, dict) else {}
    image_id = props.get("id") or feat.get("id") if isinstance(feat, dict) else None

    if image_id is None:
        print(f"[{i}/{total}] missing image id, skipping")
        continue

    image_id = str(image_id)

    # skip if already downloaded
    if any(os.path.exists(os.path.join(OUT_DIR, f"{image_id}{ext}")) for ext in [".jpg", ".jpeg", ".png", ".webp"]):
        continue

    meta_url = f"https://graph.mapillary.com/{image_id}?fields=thumb_2048_url,thumb_1024_url"

    # metadata fetch with small retry/backoff for 429
    for attempt in range(5):
        r = session.get(meta_url, timeout=30)
        if r.status_code == 429:
            wait = 1.5 * (attempt + 1)
            print(f"[{i}/{total}] 429 rate limit (meta), sleeping {wait:.1f}s")
            time.sleep(wait)
            continue
        r.raise_for_status()
        meta = r.json()
        break
    else:
        print(f"[{i}/{total}] {image_id}: failed to get metadata after retries")
        continue

    img_url = meta.get("thumb_2048_url") or meta.get("thumb_1024_url")
    if not img_url:
        print(f"[{i}/{total}] {image_id}: no thumbnail url")
        continue

    img = session.get(img_url, timeout=60)
    if img.status_code == 429:
        print(f"[{i}/{total}] 429 rate limit (img), sleeping 2s then skip")
        time.sleep(2)
        continue
    img.raise_for_status()

    ext = get_ext_from_url(img_url)
    out_path = os.path.join(OUT_DIR, f"{image_id}{ext}")

    with open(out_path, "wb") as out:
        out.write(img.content)

    if i % 100 == 0 or i == total:
        print(f"Downloaded {i}/{total}")

    time.sleep(0.05)

Downloaded 100/43081
Downloaded 200/43081
Downloaded 300/43081
Downloaded 400/43081
Downloaded 500/43081
Downloaded 600/43081
Downloaded 700/43081
Downloaded 800/43081
Downloaded 900/43081
Downloaded 1000/43081
Downloaded 1100/43081
Downloaded 1200/43081
Downloaded 1300/43081
Downloaded 1400/43081
Downloaded 1500/43081
Downloaded 1600/43081
Downloaded 1700/43081
Downloaded 1800/43081
Downloaded 1900/43081
Downloaded 2000/43081
Downloaded 2100/43081
Downloaded 2200/43081
Downloaded 2300/43081
Downloaded 2400/43081
Downloaded 2500/43081
Downloaded 2600/43081
Downloaded 2700/43081
Downloaded 2800/43081
Downloaded 2900/43081
Downloaded 3000/43081
Downloaded 3100/43081
Downloaded 3200/43081
Downloaded 3300/43081
Downloaded 3400/43081
Downloaded 3500/43081
Downloaded 3600/43081
Downloaded 3700/43081
Downloaded 3800/43081
Downloaded 3900/43081
Downloaded 4000/43081
Downloaded 4100/43081
Downloaded 4200/43081
Downloaded 4300/43081
Downloaded 4400/43081
Downloaded 4500/43081
Downloaded 4600/430

HTTPError: 504 Server Error: Gateway Timeout for url: https://scontent-bru2-1.xx.fbcdn.net/m1/v/t6/An9MrLLp2nyZXOzUpRo305vX2Wm-LhR_Ys6TM-2YoK7NQEuUUJkUS2fWQqcJ3ye7FhSksW6VwB4KoHgXrE7wEZ2BfO9DMEpas0caYSi0HMbwhj3tiaCq0OV1o3V_1M7kRZQCE10T3NLrhVr3DkFmlAo?stp=s2048x1536&edm=ALXxkZ8EAAAA&_nc_gid=Bi0sT7Tcs5vwFktIskFaXw&_nc_oc=AdndHxeB4qhbhczB2fXyOtpCyrZqrHJlZx_kRXx4d03aj0_BPRxbdE3R4MUcNjCugmw&ccb=10-6&oh=00_AfrmOt_gZBuxSQUxuIkdeslnf_tfDWX3yIrNtXr5yPJ6-Q&oe=69A2C2DD&_nc_sid=201bca